## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import q_key

# Configure gmaps API key
gmaps.configure(api_key=q_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Asau,46.4333,26.4000,27.52,96,96,2.17,overcast clouds
1,1,Zhangye,38.9342,100.4517,42.30,26,0,4.85,clear sky
2,2,Beringovskiy,63.0500,179.3167,13.17,73,20,12.95,few clouds
3,3,Mogadishu,2.0371,45.3438,80.47,83,75,9.22,broken clouds
4,4,Yellowknife,62.4560,-114.3525,-17.88,76,40,10.36,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip"))
max_temp = float(input("What is the maximum temperature you would like for your trip"))

What is the minimum temperature you would like for your trip75
What is the maximum temperature you would like for your trip90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
3,3,Mogadishu,2.0371,45.3438,80.47,83,75,9.22,broken clouds
7,7,Meulaboh,4.1363,96.1285,85.23,62,61,4.92,broken clouds
11,11,Rikitea,-23.1203,-134.9692,76.17,76,73,12.41,broken clouds
13,13,Carnarvon,-24.8667,113.6333,80.67,65,0,12.66,clear sky
16,16,Atuona,-9.8000,-139.0333,78.67,73,4,18.52,clear sky
18,18,Srandakan,-7.9386,110.2506,81.97,77,77,13.47,broken clouds
24,24,Hithadhoo,-0.6000,73.0833,82.44,76,15,11.54,few clouds
26,26,Butaritari,3.0707,172.7902,83.08,74,95,13.87,light rain
31,31,Vaini,-21.2000,-175.2000,82.56,89,20,10.36,light rain
35,35,Faanui,-16.4833,-151.7500,81.54,68,24,18.39,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             45
City                   45
Lat                    45
Lng                    45
Max Temp               45
Humidity               45
Cloudiness             45
Wind Speed             45
Weather Description    45
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,Mogadishu,80.47,broken clouds,2.0371,45.3438,
7,Meulaboh,85.23,broken clouds,4.1363,96.1285,
11,Rikitea,76.17,broken clouds,-23.1203,-134.9692,
13,Carnarvon,80.67,clear sky,-24.8667,113.6333,
16,Atuona,78.67,clear sky,-9.8000,-139.0333,
18,Srandakan,81.97,broken clouds,-7.9386,110.2506,
24,Hithadhoo,82.44,few clouds,-0.6000,73.0833,
26,Butaritari,83.08,light rain,3.0707,172.7902,
31,Vaini,82.56,light rain,-21.2000,-175.2000,
35,Faanui,81.54,few clouds,-16.4833,-151.7500,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": q_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

{'html_attributions': [], 'next_page_token': 'Aap_uEDE9liosfutIMfS_ikMYbS8_wQdZ-LLqOwBFSBr1rLptBAWyVdPqxsyL5iQgDmm24UAx8RsC5oqnlwOHXODqmGeHZpXoeE7DUkV7WInylCIM_WPgU_zqsxDj8sgNTnMbluAffU4RJekJNsdwdVwUTRRaMZoTPSt-eYb0Vkjo_OpnhGKbFpnmWnMFY15rErq6sS_hPqZlfgQY30gO8PxZqN-BYIV8UXmKQUBmyfhapRB-516677No6Yp-s00O6S1RT9Oy02UEV-9d0msXqrG7weNnmRLt43Du9vhs8n5Nu5LIS8Lueyp3gxGVrktjNaXXSDC6aEJwExoy-Iv7v4SB9RiFNBzw_eiG7Pul_ASAXEKXFbD9nL0mzl-kI3Cl2NJF24fDWaY4AP1DKrPzM9HehQLFuYccax9j9XELemIHRy6yGILEEbAc0s', 'results': [{'business_status': 'CLOSED_TEMPORARILY', 'geometry': {'location': {'lat': 2.0397296, 'lng': 45.3480703}, 'viewport': {'northeast': {'lat': 2.041267730291502, 'lng': 45.34936243029151}, 'southwest': {'lat': 2.038569769708498, 'lng': 45.34666446970851}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Juba', 'perma

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.1206197, 'lng': -134.9687529}, 'viewport': {'northeast': {'lat': -23.1194598197085, 'lng': -134.9673886697085}, 'southwest': {'lat': -23.1221577802915, 'lng': -134.9700866302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'People ThankYou', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2304, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111098332724302207930">luigia b.</a>'], 'photo_reference': 'Aap_uEB4kScl14e2kmgaQTQtppbM2_yYps7fqLWeGtru2ymy9xUYrqGFDu9Yfl4BGVuDgwot3z6dgTHBWVvcvkBr2LYFsG78Wze2gQdKe4DVWKnLIekEgpixXQj_ycWf3a9YG-PnoNh5NBUUTbd52xbEFdwJwmWo9CUjfO2mhGZ2GjrGQs7Q', 'width': 4096}], 'place_id': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'plus_code': {'global_code': '5

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -9.7952289, 'lng': -139.0305256}, 'viewport': {'northeast': {'lat': -9.793859019708496, 'lng': -139.0290940697085}, 'southwest': {'lat': -9.7965569802915, 'lng': -139.0317920302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Villa Enata', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1364, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107138465294965223459">Villa Enata</a>'], 'photo_reference': 'Aap_uEC6o84RAA1WFAkcxpwUbSKv9Zb-Uu3dT9L6vsGcjecCo76sohg7o9RH17tn6sVslclOA0BOGXDG8cSYLtxUOLZyylMKolyEEKJzGyMKagfnYiIqC8hOk9pfGUafUf90HCvEbW2K89gHl1nR5j0qEFIlYmxd055Fu7odihoMiL4rqadN', 'width': 2048}], 'place_id': 'ChIJ6TdkUOUhN3YRm5q1WA7YE4U', 'plus_code': {'compound_code': '6X

{'html_attributions': [], 'next_page_token': 'Aap_uEDvpGVsFTd4PyctXY5PFskONo6nwZkDW6lAnVcCIohUmuQxxQ63tCMKc9nz2dVSjuFktoARQrPf5ZmvjjA04wgjZi2JHelNIChvBzNevbv2B1N5vKiCujHAo6Kui8Raq0EsnUn-FqIqVxhQG1AERTh5O4gRYnhQo0Qma9s20tTyAagDXOa7BRiJbO0htIKMlOfL1s01ZlKkVm9J0L6XjbmrI5pR0jj9MYcLGR2ZBawB2EjW1oSzRysWyJN_EuxEZ6dxSTtDq3vjNTr35-8UMlYiqDxCN-uP6LFNCSWuJ6n37rg_UD27ZBAIRIUcW07RTu17hY009rhVydhTz-DhB_YmDHrkBWDLkj2L6XANcWdCg7yi7ohx-tgJ5pejxtqoiNhFdEtRwL9BJNulKKXXOer3tDBnGYaJviPS9hclhyL0F3GcdvrpOSE', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -0.608686, 'lng': 73.0853757}, 'viewport': {'northeast': {'lat': -0.607306669708498, 'lng': 73.08671053029151}, 'southwest': {'lat': -0.6100046302915021, 'lng': 73.0840125697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Scoop Guest House', 'pho

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -21.2036689, 'lng': -175.2324594}, 'viewport': {'northeast': {'lat': -21.2021358697085, 'lng': -175.2304372697085}, 'southwest': {'lat': -21.2048338302915, 'lng': -175.2331352302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Keleti Beach Resort', 'photos': [{'height': 2988, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112334378112099324058">sung ja Cho</a>'], 'photo_reference': 'Aap_uEBvCPTygIMp8wUePw99_ttgkB6GAYW1_t5K_wXMlAUGAUmSZR6m8iYzRk0_a4s6QfvT27gyT9QFZd1BMy06u9d3Gx4bv-4JO_v9f7alfSTWbkggjj5ziuDe7xT1k6nDYbtdZgDDvypuxCedM019MyVw91sOhgFjLwG963msuZYW9vOp', 'width': 5312}], 'place_id': 'ChIJA6iPT9ii9HERerEmrph0wuw', 'rating': 4, 'reference': 'ChIJA6iPT9ii9HERerEmrph0wuw', 'scop

...

{'html_attributions': [], 'next_page_token': 'Aap_uEDPcZMqCJUipAiu3X33dJhgwBgz-QvpokGdcB8KlFm6Y3q-Udy6XSMUiEJQ2Vb_CUvUE5S4yHOYh-SBpK6a7wUiwPkM8ywuT2qaRI6VoxXgVvOjl6JNaI8GMLXTcNDj2J01ahwb0tNrZ5ouiRbA6uCTekkLFO04JxypxVobn1WZ4ndE7HHJ4zAkXtCkOqvTWNS3mb-axzYE7B0rb80PGXVqwClPPQY7CWH5B1nYVGB_3YW8M_CCAfHHW51Z2momDBZGYODglcEk095-dDYuqSW4tEapRX8lhOnEFrl0zInbaFHz5fJJBzblAyEgdsCJRvm6Vqppe0kbn83YXQ-M8dg_zpUIALHZrPiXnxJxU9CSbpfo7xjUosB4tARdCFtENBPOSxwLWE2_S2oFAIU2orPG14gQAtuRnWF1g0-2o-LD2EwGIDk49HU', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -21.3403342, 'lng': 55.4727123}, 'viewport': {'northeast': {'lat': -21.3390070697085, 'lng': 55.47413358029149}, 'southwest': {'lat': -21.3417050302915, 'lng': 55.47143561970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Lindsey Hôtel', 'opening

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -2.5824468, 'lng': 150.7810058}, 'viewport': {'northeast': {'lat': -2.581097819708499, 'lng': 150.7823547802915}, 'southwest': {'lat': -2.583795780291502, 'lng': 150.7796568197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Nusa Island Retreat', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1840, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101226187795935857604">Connors Megok</a>'], 'photo_reference': 'Aap_uEDv_qbTX1GeCl8qyiW1v3W-i4Ab0PYCDHQlfYVmldqNiVP1_0sPEHLa3HY4aW3nv4y44laHRpjQ5-ulvQmSoSojP6_YzNxGEFYSrmvbLrSgLHjsPQ_78S_y-NjJYeUWTddlD5iEmGhauTtzr3jHhkqy_g71xA15QpFuv4p79O8hd5ne', 'width': 3984}], 'place_id': 'ChIJR5hSaj2wl2gRCifWWb9N3ZA', 'plus_code': {'compound_c

{'html_attributions': [], 'next_page_token': 'Aap_uECD1JVx6OneQhv9gaSuhSSzkLgjW-DbDB9MojiNNyfdeUfX65C9x37B8CJCPWVkdmVw95HMnRs3PGw7nJXqOVtlP1_soYsr1R3-AP9cCDxGxMk1-VymHn1dp-wsKkemeeneZcypJEued33KT-toSvvTH9Jwv6yrp7dBMxbdFydc8-8kl9l18UU4FhIS0zZAcaDicA0nSLaTYpWQzbXEr5kOT90PXNo8KO7CFrBnWVxVCKKNBqv6rCmYfGapcjdam-n44Zx2YUYZL5Sz_IBEZJbpfKEkTHn_Xq6UJy5LdQOnawMS0sX5dXkBoeGHoR8YzjtWhjKMoQSi0j03tZ78icTECms7JTEfw-VjBjaR3vaF5-Eh3S9FPth5VaUqLAY0hIx4FnDnYivfLHV1q9K006gspHrbNs93uanBPV0uxB_-_8m6Rqrc0ow', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 22.8809379, 'lng': -109.9117967}, 'viewport': {'northeast': {'lat': 22.8823916302915, 'lng': -109.9106242697085}, 'southwest': {'lat': 22.8796936697085, 'lng': -109.9133222302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Tesoro Los Cabos'

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.3484693, 'lng': 47.60643059999999}, 'viewport': {'northeast': {'lat': -23.3471049197085, 'lng': 47.6077222802915}, 'southwest': {'lat': -23.3498028802915, 'lng': 47.6050243197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Fanilo', 'place_id': 'ChIJNU9GQIilxCERUv0oBc6IFKo', 'plus_code': {'compound_code': 'MJ24+JH Vangaindrano, Madagascar', 'global_code': '5H89MJ24+JH'}, 'rating': 3.5, 'reference': 'ChIJNU9GQIilxCERUv0oBc6IFKo', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'Antaninarenina, Vangaindrano'}], 'status': 'OK'}
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat'

{'html_attributions': [], 'next_page_token': 'Aap_uEDCEz6iD1GKcT37lnL81BD3fOfxaCTxWj1rphyZY-I6Sgw0lP83xSgv137_ILlPkrggSebub9Jtp7J3ccC4FjF8JNKT0Qrr41ph4c6770jeDJg3xU5GsHm0CsrBGWTZxm6PPH059IcYCIGQDSULbrzr4SEUsa-KXuz9k390MI3HkLCschYeihwJx2N9Gjd2eF0auy9hWKG3gYdtAcAxUAHQmEAzZy_B-VKxdqr_82ufT7w7KuzzyREi4nlQOi5FtGp7euGst1Z6eD12ZEK9n53DKPymf805Rv4Abya4yMDfA2-LJOYAR9cZ8XEcO4lrBEGiH5iU-ESE50_pWjcEY5nXKmooD_KRp8eloxd583e7nQ_sjsboAZaU01VLU5S562Hkd9IhGd6Hzs456jTPyfEqTjHa_Jjrhl_DEmZyxK_XOL11pFWZMxA', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 16.0741237, 'lng': 78.870719}, 'viewport': {'northeast': {'lat': 16.0754682802915, 'lng': 78.87222098029152}, 'southwest': {'lat': 16.0727703197085, 'lng': 78.8695230197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Vasavi Satram', 'photos': [{'

In [11]:
hotels_df


NameError: name 'hotels_df' is not defined

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'clean_hotel_df' is not defined

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
